# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[ 4.6749854,  8.213466 , -9.075721 ,  9.568374 ,  8.454807 ,
        -1.2327975,  3.3903713, -7.8282413, -0.8454461,  0.6288572],
       [-4.243999 ,  5.610707 , -5.6697764, -1.7957243, -9.255529 ,
         0.7177438,  4.4435897, -2.874715 , -5.0900965,  9.684122 ],
       [ 5.2615476, -4.0487256,  4.464928 , -2.9367518,  3.5061095,
        -4.016832 , -3.463885 ,  6.078449 , -6.9533257, -1.004144 ],
       [-3.008261 ,  4.625961 , -4.483249 ,  2.228457 ,  1.643532 ,
        -2.4505193, -5.258201 , -1.6679403, -7.9857535,  2.8311467],
       [-5.6072407,  2.2695985, -3.7516537, -1.8182003, -5.1430283,
         7.599363 ,  2.8252366,  8.773042 ,  1.6198314,  1.1772047]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

2022-05-16 13:38:28,763 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-16 13:38:28,836 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 45.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41097,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 45.79 GiB
Comm: tcp://127.0.0.1:38555,Total threads: 1
Dashboard: http://127.0.0.1:35565/status,Memory: 22.89 GiB
Nanny: tcp://127.0.0.1:34641,


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

2022-05-16 13:38:33,765 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-c5fc61e2-d409-4335-8864-57f93143e9da
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_canary_method_should_not_exist_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-16 13:38:33,787 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-a5fc0bf5-e2d2-4198-b9dd-142179bffa02
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_display_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_display_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-16 13:38:33,808 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-dbcfe929-e40c-4e52-bb35-048ae69c446b
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception

2022-05-16 13:38:33,988 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-2aacc3ab-5ec6-4912-a19a-288978327cc7
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_canary_method_should_not_exist_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-16 13:38:34,011 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-048d61c6-e977-4989-8dce-356f0efa580e
Function:  _get_model_attr
args:      (KMeansMG(), '_repr_pdf_')
kwargs:    {}
Exception: 'AttributeError("Attribute _repr_pdf_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-16 13:38:34,034 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-617885f7-fbe4-4db3-8a6a-60184e20b1d2
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeEr

2022-05-16 13:38:34,197 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-e3f42774-384c-45c6-8802-06f53720dfcc
Function:  _get_model_attr
args:      (KMeansMG(), '_repr_javascript_')
kwargs:    {}
Exception: 'AttributeError("Attribute _repr_javascript_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'



In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[-2.8719957 ,  4.4702854 , -4.4291744 ,  2.3982146 ,  1.7456868 ,
        -2.492674  , -5.21906   , -1.70623   , -8.130277  ,  2.6402013 ,
        -4.312218  ,  5.576193  , -5.7422023 , -1.7193272 , -9.359728  ,
         0.72390175,  4.4446826 , -2.9161723 , -4.9331355 ,  9.68974   ,
         8.391241  , -6.2390246 , -6.368295  ,  1.9610256 ,  4.1622376 ,
        -9.158494  ,  4.6112323 ,  8.802037  ,  6.856204  ,  2.2432854 ],
       [ 6.2584553 ,  9.214628  ,  8.375932  ,  9.034932  ,  7.707301  ,
        -1.0178816 , -6.2575994 ,  1.3853358 , -6.9605255 , -5.9653416 ,
         1.0682615 , -0.02890242,  2.810602  ,  1.8463548 , -8.250103  ,
         3.0578778 , -8.495707  ,  9.739371  , -7.7505846 ,  3.433176  ,
        -3.9421573 , -4.1097155 ,  2.6854286 ,  1.2847115 ,  1.0240605 ,
         5.2621164 , -1.6469141 ,  6.161966  , -6.9160676 , -9.655835  ],
       [-6.926763  , -9.769037  , -6.5095406 , -0.43609676,  6.1015596 ,
         3.7510426 , -3.9631264 ,  6.1858416 , -1

## Exporting cuML Random Forest models for inferencing on machines without GPUs

Starting with cuML version 21.06, you can export cuML Random Forest models and run predictions with them on machines without an NVIDIA GPUs. The [Treelite](https://github.com/dmlc/treelite) package defines an efficient exchange format that lets you portably move the cuML Random Forest models to other machines. We will refer to the exchange format as "checkpoints."

Here are the steps to export the model:

1. Call `to_treelite_checkpoint()` to obtain the checkpoint file from the cuML Random Forest model.

In [14]:
from cuml.ensemble import RandomForestClassifier as cumlRandomForestClassifier
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)
X, y = X.astype(np.float32), y.astype(np.int32)
clf = cumlRandomForestClassifier(max_depth=3, random_state=0, n_estimators=10)
clf.fit(X, y)

checkpoint_path = './checkpoint.tl'
# Export cuML RF model as Treelite checkpoint
clf.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

/opt/conda/envs/rapids/lib/python3.9/site-packages/cuml/internals/api_decorators.py:794: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


2. Copy the generated checkpoint file `checkpoint.tl` to another machine on which you'd like to run predictions.

3. On the target machine, install Treelite by running `pip install treelite` or `conda install -c conda-forge treelite`. The machine does not need to have an NVIDIA GPUs and does not need to have cuML installed.

4. You can now load the model from the checkpoint, by running the following on the target machine:

In [15]:
import treelite

# The checkpoint file has been copied over
checkpoint_path = './checkpoint.tl'
tl_model = treelite.Model.deserialize(checkpoint_path)
out_prob = treelite.gtil.predict(tl_model, X, pred_margin=True)
print(out_prob)

[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.9        0.1        0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.  